In [59]:
import pandas as pd
import subprocess
import marshal
import os
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

pd.options.display.max_colwidth=200

In [56]:
def p4_output_to_df(p4_G_output):
    marshal_bin=[b'{s'+i for i in p4_G_output.split(b'{s')[1:]] # first one is empty
    out=[]
    for o in marshal_bin:
        try:
            out+=[marshal.loads(o)]
        except:
            print('cannot read %s'%o)
    df=pd.DataFrame(out)
    # casting to utf:
    if df.shape[0]<5:
        print('df is empty:')
        print(df.head(10))
        return df
    df=df.stack().str.decode('utf-8').unstack()
    df.columns=df.columns.astype(str)
    # removing 0 suffix from columns ater filelog -m1 command
    df.columns=df.columns.to_series().apply(lambda c:c[:-1] if c.endswith('0') else c)  # if we ran filelog -m 1 we will get most columns with 0 suffix (this number says the output line)
    # fixing types:
    if 'time' in df.columns:
        df.time=pd.to_datetime(df.time, unit='s')
    for t in ['rev','change','fileSize']:
        if t in df.columns:
            df[t]=df[t].fillna(-1).astype(int)
    # extracting block name
    if 'depotFile' in df.columns:
        df[['library','block','block_type','file_name']]=df.depotFile.str.rsplit('/',4, expand=True).drop(0, axis=1)
    try:
        df=df.drop(['action','change','client','code','digest','fileSize','type','library'], axis=1)
    except: pass
    df=df.drop(df.columns[df.columns.str.contains(',')].values, axis=1)
    df=df.rename(columns=dict(desc='comment',depotFile='p4_path', rev='version'))
    if df.empty:
        print('df is empty!')
    return df


In [ ]:
client='iil:lisrael1:e039b3fb9db4738ff81a240e2564dbdbaec65b22:amsbase_A:gen2_tc'
port='ssl:p4brk2idce.sync.intel.com:7070'
p4_user=f'-c {client} -p {port}'
extract_sub_cells=''' | grep -Po '\("\w+"\s*"\w+"\s*"\w+"\)' | tr -d '()"' | sort | uniq -c | awk '{print $1"*"$3}' '''

block_name='ipn5adcsarshr_core'


cmd=f'p4 {p4_user} have //barak2/... | grep "/{block_name}/schematic/data.dm"'

In [37]:
# get all p4 local files
cmd=f'p4 -G {p4_user} have //barak2/...'
p4_output=subprocess.Popen(cmd.split(' '),stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout.read()
p4_output=p4_output_to_df(p4_output)
p4_output.head()

,clientFile,code,p4_path,haveRev,path,syncTime,library,block,block_type,file_name
0,//iil:lisrael1:e039b3fb9db4738ff81a240e2564dbd...,stat,//barak2/barak_gen2/A/blkProps/.p4ignore,1,/nfs/iil/disks/hip_ana_users_04/lisrael1/barak...,1568017244,barak_gen2,A,blkProps,.p4ignore
1,//iil:lisrael1:e039b3fb9db4738ff81a240e2564dbd...,stat,//barak2/barak_gen2/A/blkProps/cdn/blockProps.lib,1,/nfs/iil/disks/hip_ana_users_04/lisrael1/barak...,1568017244,A,blkProps,cdn,blockProps.lib
2,//iil:lisrael1:e039b3fb9db4738ff81a240e2564dbd...,stat,//barak2/barak_gen2/A/blkProps/cdn/env/.cdsenv,1,/nfs/iil/disks/hip_ana_users_04/lisrael1/barak...,1568017244,blkProps,cdn,env,.cdsenv
3,//iil:lisrael1:e039b3fb9db4738ff81a240e2564dbd...,stat,//barak2/barak_gen2/A/blkProps/cdn/env/.cdsinit,8,/nfs/iil/disks/hip_ana_users_04/lisrael1/barak...,1580628914,blkProps,cdn,env,.cdsinit
4,//iil:lisrael1:e039b3fb9db4738ff81a240e2564dbd...,stat,//barak2/barak_gen2/A/blkProps/cdn/env/CS-1846...,1,/nfs/iil/disks/hip_ana_users_04/lisrael1/barak...,1572426187,blkProps,cdn,env,CS-1846.ile


In [62]:
a=p4_output.query(f'path.str.contains("/{block_name}/schematic/data.dm")').apply(lambda r:f'strings {r.path}'+extract_sub_cells, axis=1)
p4_output['subcell_command']=p4_output.progress_apply(lambda r:f'strings {r.path}'+extract_sub_cells, axis=1)
p4_output['subcells']=p4_output.query(f'path.str.contains("/schematic/data.dm")').subcell_command.progress_apply(lambda r:os.popen(r).read().split('\n'))
p4_output

KeyboardInterrupt: 